In [2]:
# prepare environment
import pandas

top_companies = pandas.read_table("../data/top_companies.txt", sep='    ')
print top_companies.shape

top_companies.head()
del top_companies['Latest Filings']

(12000, 5)


/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [3]:
import urllib2
from bs4 import BeautifulSoup
import sqlite3

conn = sqlite3.connect('h1b_salary.db')
c = conn.cursor() 
c.execute("DROP TABLE salary")
c.execute('''CREATE TABLE salary
             (company_name text, job_title text, salary integer, city_name text, submit_date text, start_date text, status text)''')

# loop over top companies list
for index, row in top_companies.iterrows():
    employer = row["Company Name"].replace(" ","+")
    print employer
    url = "http://h1bdata.info/index.php?em="+employer+"&job=&city=&year=All+Years"
    soup = BeautifulSoup(urllib2.urlopen(url).read(),'html.parser')
    table = soup.find('table', attrs={'id': 'myTable'})

    rows = table.findAll('tr')

    for tr in rows:
        cols = tr.findAll('td')
        text = ""
        for td in cols:
            text = text + td.find(text=True) + ';'
            if text.count(";") == 7:
                row_to_insert = "'" + text.replace(";","','")[:-2]
                #print row_to_insert
                c.execute("INSERT INTO salary VALUES (" + row_to_insert + ")")
    conn.commit()

#for row in c.execute('SELECT * FROM salary'):
#    print row
    
conn.close()

INFOSYS+LIMITED
TATA+CONSULTANCY+SERVICES+LIMITED


OperationalError: near "FALLON": syntax error